<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    weights=[0.9, 0.1],
    flip_y=0,
    random_state=42
)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [10]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/01/03 17:58:59 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.


2025/01/03 17:59:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/1/runs/52a553d79bf0441f9817adc3f33e03cf
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025/01/03 17:59:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/1/runs/34c23bf1f4a14019a23ab37425432628
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025/01/03 17:59:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/1/runs/9c23b0ebdbbc4c6fb6dc294301c65f88
🧪 View experiment at: http://localhost:5000/#/experiments/1


2025/01/03 17:59:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/1/runs/3cbe266ff61d40929b614792a607cb8f
🧪 View experiment at: http://localhost:5000/#/experiments/1


### Register the Model

In [11]:
model_name = 'XGB_SMOTE'
run_id = input('Please type RunID')
model_uri = f'runs:/{run_id}/{model_name}'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB_SMOTE'.
2025/01/03 18:07:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_SMOTE, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/1/runs/3cbe266ff61d40929b614792a607cb8f
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '1' of model 'XGB_SMOTE'.


In [14]:
mlflow.config.get_registry_uri()

'http://localhost:5000'

In [16]:
mlflow.get_registry_uri()

'http://localhost:5000'

In [17]:
mlflow.get_tracking_uri()

'http://localhost:5000'

### Load the Model

In [22]:
import mlflow.xgboost

# Load model directly from the local path
local_model_path = "/Users/fawad/Desktop/Data science/projects/mlflow_mlops_projects/mlruns/1/3cbe266ff61d40929b614792a607cb8f/artifacts/model"
loaded_model = mlflow.xgboost.load_model(model_uri=local_model_path)

# Predict
y_pred = loaded_model.predict(X_test)
print(y_pred[:4])

[0 0 0 0]


### Transition the Model to Production

In [25]:
current_model_uri = f"models:/{model_name}/1"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB_SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1735912735458, current_stage='None', description='', last_updated_timestamp=1735912735458, name='anomaly-detection-prod', run_id='3cbe266ff61d40929b614792a607cb8f', run_link='', source='models:/XGB_SMOTE/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [30]:
# model_version = 1
# prod_model_uri = f"models:/{production_model_name}/1"

# loaded_model = mlflow.xgboost.load_model(prod_model_uri)
# y_pred = loaded_model.predict(X_test)
# y_pred[:4]

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 